<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/Train_Inceptionv3_Network_with_Sample_Labeled_data_(1000).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Environment set up

In [ ]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-0c5721cb-47cd-8675-c5bc-fc5b969c7cb2)


In [ ]:
import zipfile
import datetime, os
from random import shuffle
import tensorflow as tf
import zipfile
import itertools
import numpy as np
import h5py
import random
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mlxtend.plotting import plot_confusion_matrix 
%matplotlib inline

from keras.models import Model, Sequential, model_from_json

import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from google.colab import drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build


print(f"Numpy version: {np.__version__}")
print(f"Tensorflow version: {tf.__version__}")

random.seed(432)

Numpy version: 1.18.5
Tensorflow version: 2.3.0


In [ ]:
# Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
auth.authenticate_user()

In [ ]:
data_samples = {
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv"
}

In [ ]:
SAMPLE = 'sample@1000'
SAMPLE_NBR = 1000

URL_SAMPLE = data_samples[SAMPLE]
URL_VAL_VOETS = data_samples["val-voets-tf"]
ROOT_PATH = 'drive/My Drive/Project'

In [ ]:
!gdown $URL_SAMPLE

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/sample@1000.zip
108MB [00:00, 125MB/s]  


In [ ]:
!gdown $URL_VAL_VOETS

Downloading...
From: https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv
To: /content/val-voets-tf_records.zip
613MB [00:03, 172MB/s]


In [ ]:
local_zip = '/content/{}.zip'.format(SAMPLE)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
local_zip = '/content/{}.zip'.format("val-voets-tf_records")
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
train_dir = "{}/train".format(SAMPLE)
val_dir = "val-voets-tf_records/val"

In [ ]:
def rescale_min_1_to_1(image):
    """
    Rescale image to [-1, 1].
    :param image:
        Required. Image tensor.
    :return:
        Scaled image.
    """
    # Image must be casted to float32 first.
    image = tf.cast(image, tf.float32)
    # Rescale image from [0, 255] to [0, 2].
    image = tf.multiply(image, 1. / 127.5)
    # Rescale to [-1, 1].
    return tf.subtract(image, 1.0)

def rescale_0_to_1(image):
    """
    Rescale image to [0, 1].
    :param image:
        Required. Image tensor.
    :return:
        Scaled image.
    """
    return tf.image.convert_image_dtype(image, tf.float32)

def _tfrecord_dataset_from_folder(folder, ext='.tfrecord'):
    tfrecords = [os.path.join(folder, n)
                 for n in os.listdir(folder) if n.endswith(ext)]
    return tf.data.TFRecordDataset(tfrecords)


def _parse_example(proto, num_channels, normalization_fn, augmentation=True):
    features = {"image": tf.io.FixedLenFeature((), tf.string),
                "format": tf.io.FixedLenFeature((), tf.string),
                "label": tf.io.FixedLenFeature((), tf.int64),
                "height": tf.io.FixedLenFeature((), tf.int64),
                "width": tf.io.FixedLenFeature((), tf.int64)}
    parsed = tf.io.parse_single_example(proto, features)

    image = tf.image.decode_jpeg(parsed["image"], num_channels)
    image = normalization_fn(image)

    # Apply data augmentations randomly.
    augmentations = [
        {'fn': tf.image.random_flip_left_right},
        {'fn': tf.image.random_brightness,
         'args': [BRIGHTNESS_MAX_DELTA]},
        {'fn': tf.image.random_saturation,
         'args': [SATURATION_LOWER, SATURATION_UPPER]},
        {'fn': tf.image.random_hue,
         'args': [HUE_MAX_DELTA]},
        {'fn': tf.image.random_contrast,
         'args': [CONTRAST_LOWER, CONTRAST_UPPER]}]

    shuffle(augmentations)

    if augmentation:
        for aug in augmentations:
            if 'args' in aug:
                image = aug['fn'](image, *aug['args'])
            else:
                image = aug['fn'](image)

    # Transpose images to channels first since training is done on GPU(s)
    #  and cuDNN works more efficiently with channels in first dimension.
    
    #image = tf.transpose(image, [2, 0, 1])

    label = tf.cast(
                tf.reshape(parsed["label"], [-1]),
                tf.float32)

    return image, label

def _parse_example2(proto, num_channels, normalization_fn, augmentation=True):
    features = {"image/encoded": tf.io.FixedLenFeature((), tf.string),
                "image/format": tf.io.FixedLenFeature((), tf.string),
                "image/class/label": tf.io.FixedLenFeature((), tf.int64),
                "image/height": tf.io.FixedLenFeature((), tf.int64),
                "image/width": tf.io.FixedLenFeature((), tf.int64)}
    parsed = tf.io.parse_single_example(proto, features)

    image = tf.image.decode_jpeg(parsed["image/encoded"], num_channels)
    image = normalization_fn(image)

    label = tf.cast(
                tf.reshape(parsed["image/class/label"], [-1]),
                tf.float32)

    return image, label


def initialize_dataset(image_dir, batch_size, num_epochs=1,
                       num_workers=1, prefetch_buffer_size=None,
                       shuffle_buffer_size=None,
                       normalization_fn=tf.image.per_image_standardization,
                       num_channels=3, image_dim=[299, 299],
                       augmentation=True):
    
    # Retrieve data set from pattern.
    dataset = _tfrecord_dataset_from_folder(image_dir)

    dataset = dataset.map(
        lambda e: _parse_example(e, num_channels, normalization_fn, augmentation),
        num_parallel_calls=num_workers)

    if shuffle_buffer_size is not None:
        dataset = dataset.shuffle(shuffle_buffer_size)

    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)

    if prefetch_buffer_size is not None:
        dataset = dataset.prefetch(prefetch_buffer_size)

    return dataset


def initialize_dataset2(image_dir, batch_size, num_epochs=1,
                       num_workers=1, prefetch_buffer_size=None,
                       shuffle_buffer_size=None,
                       normalization_fn=tf.image.per_image_standardization,
                       num_channels=3, image_dim=[299, 299],
                       augmentation=True):
    
    # Retrieve data set from pattern.
    dataset = _tfrecord_dataset_from_folder(image_dir)

    dataset = dataset.map(
        lambda e: _parse_example2(e, num_channels, normalization_fn, augmentation),
        num_parallel_calls=num_workers)

    if shuffle_buffer_size is not None:
        dataset = dataset.shuffle(shuffle_buffer_size)

    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)

    if prefetch_buffer_size is not None:
        dataset = dataset.prefetch(prefetch_buffer_size)

    return dataset

In [ ]:
BRIGHTNESS_MAX_DELTA = 0.125
SATURATION_LOWER = 0.5
SATURATION_UPPER = 1.5
HUE_MAX_DELTA = 0.2
CONTRAST_LOWER = 0.5
CONTRAST_UPPER = 1.5

# Various constants.
num_channels = 3
num_workers = 8
normalization_fn = rescale_min_1_to_1

# Hyper-parameters for training.
learning_rate = 1e-6
decay = 4e-5
train_batch_size = 32

# Hyper-parameters for validation.
num_epochs = 150
wait_epochs = 10
min_delta_auc = 0.01
val_batch_size = 32
num_thresholds = 200
kepsilon = 1e-7

# Buffer size for image shuffling.
shuffle_buffer_size = 2048
prefetch_buffer_size = 2 * train_batch_size

img_width, img_height = 299, 299

# Parameters 
BATCH_SIZE = 32
img_width, img_height = 299, 299

# 1. Experimentation 

In [ ]:
# Initialize each data set.
train_dataset = initialize_dataset(
    train_dir, 
    train_batch_size,
    num_epochs=num_epochs,
    num_workers=num_workers, 
    prefetch_buffer_size=prefetch_buffer_size,
    shuffle_buffer_size=shuffle_buffer_size, 
    num_channels=num_channels,
    normalization_fn=normalization_fn)

validation_dataset = initialize_dataset2(
    val_dir, 
    val_batch_size,
    num_epochs=num_epochs,
    num_workers=num_workers, 
    prefetch_buffer_size=prefetch_buffer_size,
    shuffle_buffer_size=shuffle_buffer_size, 
    num_channels=num_channels,
    normalization_fn=normalization_fn)

In [ ]:
steps_per_epoch = SAMPLE_NBR // BATCH_SIZE # CHANGE HERE 
#validation_steps = val_generator.n // BATCH_SIZE

#### Warmup
First we do a warmup training only the top layers:

In [ ]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', 
                    include_top=False,
                    pooling='avg',
                    input_shape=(img_width, img_height,3))

87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
# Freeze the feature extractor
for layer in base_model.layers:
    layer.trainable=False

x = Dense(2048)(base_model.output)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate,
                                       decay=decay),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=355
)

Epoch 1/20
31/31 [==============================] - 26s 840ms/step - loss: 0.7135 - accuracy: 0.5000 - auc: 0.4797 - val_loss: 0.7260 - val_accuracy: 0.4424 - val_auc: 0.5252
Epoch 2/20
31/31 [==============================] - 23s 736ms/step - loss: 0.7013 - accuracy: 0.5101 - auc: 0.5247 - val_loss: 0.7060 - val_accuracy: 0.4960 - val_auc: 0.5363
Epoch 3/20
31/31 [==============================] - 23s 734ms/step - loss: 0.7019 - accuracy: 0.5373 - auc: 0.5200 - val_loss: 0.6908 - val_accuracy: 0.5411 - val_auc: 0.5476
Epoch 4/20
31/31 [==============================] - 23s 737ms/step - loss: 0.6972 - accuracy: 0.5323 - auc: 0.5286 - val_loss: 0.6923 - val_accuracy: 0.5351 - val_auc: 0.5553
Epoch 5/20
31/31 [==============================] - 23s 732ms/step - loss: 0.6982 - accuracy: 0.5171 - auc: 0.5273 - val_loss: 0.6962 - val_accuracy: 0.5246 - val_auc: 0.5624
Epoch 6/20
31/31 [==============================] - 23s 733ms/step - loss: 0.6912 - accuracy: 0.5544 - auc: 0.5591 - val_loss

In [ ]:
# Serialize model to JSON
model_json = model.to_json()
with open('{}/{}/{}'.format(ROOT_PATH, "model", "InceptionV3_warm_up_{}.json".format(SAMPLE)), "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
model.save_weights('{}/{}/{}'.format(ROOT_PATH, "model", "InceptionV3_warm_up_{}.h5".format(SAMPLE)))
print("Saved model to disk")

In [ ]:
#[OPTIONAL] Resume training from saved weigths
resume_json_file_structure = "InceptionV3_warm_up_{}.json".format(SAMPLE)

# Load json file structure and create model
json_file = open('drive/My Drive/Project/model/{}'.format(resume_json_file_structure), 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# Load weights into new model
resume_model_weigths = 'InceptionV3-sample@1000-epochs:043-val_loss:0.585.hdf5' # CHANGE THIS 
model.load_weights('drive/My Drive/Project/model/{}'.format(resume_model_weigths))
print("Loaded model from Drive")

Loaded model from Drive


#### Unfreeze the extractor layers

In [ ]:
for layer in model.layers:
    layer.trainable = True

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate,
                                       decay=decay),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

file_name = "InceptionV3-{}".format(SAMPLE)+"-epochs:{epoch:03d}-val_loss:{val_loss:.3f}.hdf5"
filepath = '{}/{}/{}'.format(ROOT_PATH, "model",  file_name)

checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only = True, 
                             mode='auto')

early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0, 
                           patience=200, 
                           verbose=1, 
                           mode='auto', 
                           baseline=None, 
                           restore_best_weights=True)

In [ ]:
# Train
history = model.fit(
    train_dataset,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=355,
    callbacks = [checkpoint, early_stop]
)

Epoch 1/150
31/31 [==============================] - ETA: 0s - loss: 0.4755 - accuracy: 0.8125 - auc_1: 0.8795
Epoch 00001: val_loss improved from inf to 0.58741, saving model to drive/My Drive/Project/model/InceptionV3-sample@1000-epochs:001-val_loss:0.587.hdf5
31/31 [==============================] - 28s 899ms/step - loss: 0.4755 - accuracy: 0.8125 - auc_1: 0.8795 - val_loss: 0.5874 - val_accuracy: 0.6916 - val_auc_1: 0.6886
Epoch 2/150
31/31 [==============================] - ETA: 0s - loss: 0.4633 - accuracy: 0.7974 - auc_1: 0.8917
Epoch 00002: val_loss did not improve from 0.58741
31/31 [==============================] - 25s 822ms/step - loss: 0.4633 - accuracy: 0.7974 - auc_1: 0.8917 - val_loss: 0.6021 - val_accuracy: 0.6746 - val_auc_1: 0.6882
Epoch 3/150
31/31 [==============================] - ETA: 0s - loss: 0.4647 - accuracy: 0.8004 - auc_1: 0.8869
Epoch 00003: val_loss did not improve from 0.58741
31/31 [==============================] - 25s 805ms/step - loss: 0.4647 - accu